In [1]:
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import math
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import random
import matplotlib.pyplot as plt

In [2]:
filename = '/content/drive/My Drive/annotedData.csv'
data = pd.read_csv(filename)
file = open(filename, 'r')
raw_text = file.read()
file.close()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/annotedData.csv'

In [ ]:
def to_Int(x):
    s = 0
    for i in x:
        if(i >= '0' and i <= '9'):
            s *= 10
            s += int(i)
    return s


In [ ]:
data.dropna(inplace = True)

data = data[:int(99*len(data)/100)]

data.Sent = data.Sent.apply(to_Int)

data.tail()

In [ ]:
#Generates a list of tuples of (tweets, character wise classification whether the character is in an NER or not)

inp_tweets = []

val = 0
j = 0
for val in range(1521):
    s1 = ""
    s2 = ""
    while(int(data['Sent'].iloc[j]) == val):
        s1 += str((data['Word']).iloc[j])
        s1 += " "
        if(str(data['Tag'].iloc[j]) == 'Other'):
            c = '0'
        else:
            c = '1'
        for y in range(len(data['Word'].iloc[j])):
            s2 += c
        s2 += '0'
        j += 1
    assert(len(s1) == len(s2))
    if(s1 != ''):
        inp_tweets.append([s1, s2])

inp_tweets[0]

In [ ]:
#Test-Train split 80-20

EIGHTY_TWENTY = int(80/100 * len(inp_tweets))
train = inp_tweets[:EIGHTY_TWENTY]
test = inp_tweets[EIGHTY_TWENTY:]

In [ ]:
#Making a mapping of characters for one-hot encoding
char_list = ['^'] + sorted(set(raw_text))
mapping = dict((c, i) for i, c in enumerate(char_list))
N_CHARS = len(char_list)

In [ ]:
x_train = [x[0] for x in train]
y_train = [x[1] for x in train]
x_test = [x[0] for x in test]
y_test = [x[1] for x in test]

In [ ]:
#Util functions to create tensors of one-hot encoding of characters

def letter_to_index(letter):
    return mapping[letter]

#Generates a 1xn tensor.
def letter_to_tensor(letter):
    tensor = torch.zeros(N_CHARS)
    tensor[letter_to_index(letter)] = 1
    return tensor

#Generates a line_lengthx1xn tensor
def line_to_tensor(line):
    tensor = torch.zeros(len(line), N_CHARS)
    for i, letter in enumerate(line):
        tensor[i][letter_to_index(letter)] = 1
    return tensor

def output_to_tensor(line):
  tensor = torch.zeros(len(line))
  for i, letter in enumerate(line):
    tensor[i] = int(letter)
  return tensor

In [ ]:
#Pad all strings to max tweet length for RNN training

print(f'Tweet before padding {x_train[0]}')
print(f'Answer before padding {y_train[0]}\n')

for i in range(len(x_train)):
    x_train[i] = x_train[i].ljust(MAX_TWEET_LEN, '^')
    y_train[i] = y_train[i].ljust(MAX_TWEET_LEN, '0')

for i in range(len(x_test)):
    x_test[i] = x_test[i].ljust(MAX_TWEET_LEN, '^')
    y_test[i] = y_test[i].ljust(MAX_TWEET_LEN, '0')


In [ ]:
#Convert strings to list of one-hot encodings of characters
str_x_test = x_test.copy()

for i in range(len(x_train)):
    x_train[i] = line_to_tensor(x_train[i])
    y_train[i] = output_to_tensor(y_train[i])

for i in range(len(x_test)):
    x_test[i] = line_to_tensor(x_test[i])
    y_test[i] = output_to_tensor(y_test[i])

x_train[:2]
y_train[:2]

In [ ]:
#Convert lists to tensors
x_train = torch.stack(x_train)
y_train = torch.stack(y_train)

x_test = torch.stack(x_test)
y_test = torch.stack(y_test)

In [ ]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
#Define bi-directional RNN module
class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super(GRU, self).__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.input_size = input_size
    self.sigmoid = nn.Sigmoid()

In [ ]:
model = GRU(x_train.shape[2], 128, 1)
# input_tensor = line_to_tensor('Cat is named Kalyan')
# input_tensor = input_tensor.view(1, -1, N_CHARS)

# out_tensor = torch.zeros(1, 19)
# out = model(input_tensor)
# print(loss_function(out, out_tensor))

In [ ]:
train = [(x_train[i], y_train[i]) for i in range(len(x_train))]
test = [(x_test[i], y_test[i]) for i in range(len(x_test))]

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test, batch_size = batch_size, shuffle = True)

In [ ]:
#Train
loss_function = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
  loss_g = 0
  for i, (inp, out) in enumerate(train_loader):
    output = model(inp)
    loss = loss_function(output, out)

In [ ]:
#Accuracy of words from test-set
with torch.no_grad():
  ner_correct, nner_correct, ner_wrong, nner_wrong = 0, 0, 0, 0
  for i in range(len(str_x_test)):
    tens = line_to_tensor(str_x_test[i]).view(1, -1, N_CHARS)
    tens = model(tens

In [ ]:
print(f'The accuracy of the model on the test set is: {overall_accuracy:.2f}%')
print(f'The accuracy of the model on predicting Named Entities is: {ner_accuracy:.2f}%')
print(f'The accuracy of the model on predicting Non-Named Entities is: {nner_accuracy:.2f}%')
